<a href="https://colab.research.google.com/github/ihshin227/tpu/blob/master/Keras_LSTM_and_RNN_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(4, return_sequences=True))
model.add(tf.keras.layers.SimpleRNN(4))
model.add(tf.keras.layers.Dense(units=1, input_shape=[32,4]))
model.compile(optimizer='sgd', loss='mean_squared_error')

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.LSTM(4, return_sequences=True))
# model.add(tf.keras.layers.LSTM(4))
# model.add(tf.keras.layers.Dense(units=1, input_shape=[32,4]))
# model.compile(optimizer='sgd', loss='mean_squared_error')

In [0]:
inputs = np.random.random([32, 18, 18]).astype(np.float32)
targets = np.random.rand(32).astype(np.float32)

model.fit(inputs, targets, epochs=20)

Epoch 1/20
1/1 [==============================] - 0s 1ms/step - loss: 0.8218
Epoch 2/20
1/1 [==============================] - 0s 4ms/step - loss: 0.1968
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1791
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1700
Epoch 5/20
1/1 [==============================] - 0s 2ms/step - loss: 0.1640
Epoch 6/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1594
Epoch 7/20
1/1 [==============================] - 0s 996us/step - loss: 0.1555
Epoch 8/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1519
Epoch 9/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1485
Epoch 10/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1454
Epoch 11/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1424
Epoch 12/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1395
Epoch 13/20
1/1 [==============================] - 0s 1ms/step - loss: 

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [0]:
tflite_model = converter.convert()

In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

In [0]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [0]:
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [0]:
interpreter.invoke()

In [0]:
tflite_result = interpreter.get_tensor(output_details[0]['index'])

In [0]:
tf_results = model(tf.constant(input_data))

In [0]:
for tf_r, tflite_r in zip(tf_results, tflite_result):
  np.testing.assert_almost_equal(tf_r, tflite_r, decimal=5)
  print(tf_r, tflite_r)

tf.Tensor([0.06198855], shape=(1,), dtype=float32) [0.06198839]
tf.Tensor([0.5964721], shape=(1,), dtype=float32) [0.59647214]
tf.Tensor([0.94845533], shape=(1,), dtype=float32) [0.94845533]
tf.Tensor([0.84689415], shape=(1,), dtype=float32) [0.8468942]
tf.Tensor([0.4987081], shape=(1,), dtype=float32) [0.4987081]
tf.Tensor([0.68650234], shape=(1,), dtype=float32) [0.68650246]
tf.Tensor([0.93921655], shape=(1,), dtype=float32) [0.93921643]
tf.Tensor([0.8302045], shape=(1,), dtype=float32) [0.8302043]
tf.Tensor([1.0425596], shape=(1,), dtype=float32) [1.0425596]
tf.Tensor([0.4070053], shape=(1,), dtype=float32) [0.40700546]
tf.Tensor([0.9470096], shape=(1,), dtype=float32) [0.94700974]
tf.Tensor([1.1175016], shape=(1,), dtype=float32) [1.1175017]
tf.Tensor([0.75643617], shape=(1,), dtype=float32) [0.75643617]
tf.Tensor([0.43131533], shape=(1,), dtype=float32) [0.4313154]
tf.Tensor([0.93861514], shape=(1,), dtype=float32) [0.9386149]
tf.Tensor([0.96563256], shape=(1,), dtype=float32) [0.

In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     multiple                  92        
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     multiple                  36        
_________________________________________________________________
dense_4 (Dense)              multiple                  5         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [0]:
tflite_model.summary()